<div align="center" dir="auto">
<p dir="auto">

<a href="https://colab.research.google.com/github/write-with-neurl/modelbit-articles/blob/main/modelbit-03/code/Deploy_RESNET-50_Model_With_Modelbit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

</p>

# ⚡ Deploying ResNet-50 Model to A Rest API Endpoint for Image Classification | Modelbit

## 🧑‍💻 Installations and Set Up

Login.

In [1]:
# Using latest version of pip
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
# Install the latest version of 'modelbit' for model deployment quietly
!pip install -q --upgrade modelbit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.1 MB/s eta 0:00:00


Install libraries you will use to build and test the ResNet-50 model.

In [3]:
import requests
import json
import numpy as np
import matplotlib.pyplot as plt

import torch

from torchvision import models, transforms
from PIL import Image
from io import BytesIO

device = torch.device("cpu")

## 🔃 Load the pretrained model from pytorch to memory

In [4]:
# Load the pretrained model from pytorch
resnet50 = models.resnet50(pretrained=True)
resnet50 = resnet50.to(device)
resnet50.eval() # Set the model to evaluation mode

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 128MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### 🏷️ Download ImageNet labels

In [5]:
# Download ImageNet labels
LABELS_URL = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'

# Load them to a variable
labels = requests.get(LABELS_URL).json()


### 🖼️ Download Sample Images

In [6]:
# Downloading images in quiet mode (without showing download progress )
# Download a picture of a dog and cat.jpg
!wget -q -O dog_and_cat.jpg http://doc.modelbit.com/img/cat.jpg

## 🧪 Test your sample image with the ResNet-50 model

In [7]:
# Here we are loading an image from disk
image_path = "dog_and_cat.jpg"
img = Image.open(image_path)

# Preprocess the image
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Pass the image for preprocessing and reshape to add batch dimension
input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0)

# Predict the class of the image
with torch.no_grad():
    output = resnet50(input_batch)

_, predicted_idx = torch.max(output, 1)
print(f"Predicted class index: {predicted_idx.item()}")
print(f"Predicted class label: {labels[predicted_idx.item()]}")


Predicted class index: 282
Predicted class label: tiger cat


## 📸 Use a helper function to display the ResNet-50 model prediction and label

In [8]:
def display_image(inp, input_batch, predicted_label):
    # De-normalize the image for display
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = input_batch[0].numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    fig = plt.figure(figsize=(16,4))
    plt.title(predicted_label)
    plt.imshow(inp)

    plt.axis('off')
    return fig

In [ ]:
display_image(img, input_batch, predicted_label=labels[predicted_idx.item()])

## 🚀 Deploying Model to a REST API Endpoint

### 🔐 Log into `modelbit`

Use the [`modelbit`](https://doc.modelbit.com/deployments/) library for model deployment and management. Modelbit offers you a flexible approach to [deploy ML models](https://www.modelbit.com/product/deploy-from-anywhere) from your Colab or Jupyter notebooks, or any Python environment, to production environments with REST APIs. With fully custom Python environments. Backed by your git repo.

In [10]:
import modelbit

# Log into the 'modelbit' service using the development ("dev") branch
# Ensure you create a "dev" branch in Modelbit or use the "main" branch for your deployment
mb = modelbit.login(branch="dev")

To deploy your ResNet-50 model to a REST API Endpoint on Modelbit, you will need to bundle the prediction logic that:
- Loads the image you wthe model
- Preprocesses that image
- Transforms the image to a tensor
- And passes the image to the ResNet-50 model for classification.

Essentially purposing the code you use to test the model!

### 🛠️ Define the function

In [11]:
def resnet_inference(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))

    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Pass the image for preprocessing and reshape to add batch dimension
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)

    # Predict the class of the image
    with torch.no_grad(): # no_grad ensures the gradients are not calculated in prod
        output = resnet50(input_batch)

    _, predicted_idx = torch.max(output, 1)

    pred_img = display_image(img, input_batch, predicted_label=labels[predicted_idx.item()])
    mb.log_image(pred_img) # show the predicted boxes on the image in modelbit logs

    return { "index": predicted_idx.item(), "label": labels[predicted_idx.item()]}

In a case where you want to visually inspect the images from requests that hit your endpoint, you can use [`mb.log_image()`](https://doc.modelbit.com/api-reference/log_image/) to log the predicted image to the modelbit platform. So you or any SME can go in and inspect the accuracy of the classification.

Calling the `mb.deploy()` API will run the following for you under-the-hood:

- **Push** the source code to your Modelbit workspace.

- **Pickle the varialbles in the dev environment**: Your function's variables are pickled, i.e., serialized into a format that can be stored and reconstructed.

- **Detect the dependencies:** Unlike traditional deployment strategies that require manual tracking of dependencies, ModelBit intelligently detects which dependencies, libraries, and data your application needs.

- **Containerize the model weights and other helper files:** It then automatically incorporates these into the container it builds, significantly reducing the possibility of errors and saving deployment time.

- **Spin up a REST API endpoint**: After containerizing your model and it’s dependencies, Modelbit replicates the environment in production for consistency and spins up a REST endpoint.

In [12]:
# This would be how you deploy to Modelbit
mb.deploy(resnet_inference)

Uploading 'resnet50': 100%|██████████| 95.0M/95.0M [00:04<00:00, 23.3MB/s]


## 📩 Test the REST Endpoint with a Single Image

Test your endpoint from the command line using:

> ⚠️ Replace the `ENTER_WORKSPACE_NAME` placeholder with your workspace name.

In [99]:
!curl -s -XPOST "https://ENTER_WORSKPACE_NAME.app.modelbit.com/v1/resnet_inference/dev/latest" -d '{"data": "https://moderndogmagazine.com/sites/default/files/images/uploads/Samoyed_1.jpg"}' | json_pp

{
   "data" : {
      "index" : 258,
      "label" : "Samoyed"
   }
}


You can also test your REST Endpoint by [sending single or batch production images](https://doc.modelbit.com/deployments/rest-api/single-inference) to it for scoring.

Use the `requests` package to POST a request to the API and use `json` to format the response to print nicely:


> ⚠️ Replace the `ENTER_WORKSPACE_NAME` placeholder with your workspace name.

In [14]:
import json

requests.post("https://ENTER_WORSKPACE_NAME.app.modelbit.com/v1/resnet_inference/dev/latest",
              headers={"Content-Type":"application/json"},
              data=json.dumps({"data": "https://moderndogmagazine.com/sites/default/files/images/uploads/Samoyed_1.jpg"})).json()


{'data': {'index': 258, 'label': 'Samoyed'}}

# 📚 Modelbit Machine Learning Blog

Enjoyed this walkthrough? Check out articles with similar walkthroughs:

- [Deploying a Grounding DINO Model to a REST API Endpoint for Open-Set Object Detection with Prompts](https://www.modelbit.com/blog/deploying-grounding-dino-model-to-a-rest-api-endpoint-for-open-set-object-detection-with-prompts)

- [Deploying a Segment-Anything Image Recognition Model to a REST Endpoint](https://www.modelbit.com/blog/deploying-a-segment-anything-image-recognition-model-to-a-rest-endpoint)